In [1]:
import requests
import pandas as pd

In [3]:
sdc_dfs = []
offset = 0
limit = 1000
while True:
    api = f"https://data.usgs.gov/datacatalog/api/dcat/?start_date=1960-01-01&end_date=2023-12-31&page={offset}&size={limit}"
    r = requests.get(api)
    if r.status_code != 200:
        break
    data = r.json()
    sdc_dfs.append(pd.DataFrame(data['dataset']))
    print(offset, len(data['dataset']))
    offset+=limit


KeyboardInterrupt: 

In [7]:
sdc_dcat = pd.concat(sdc_dfs)

In [22]:
sdc_unique = sdc_dcat.drop_duplicates(subset=['identifier','title','description'], keep='first').reset_index(drop=True)
sdc_unique.drop(columns=['@type','bureauCode','theme','accessLevel','publisher'], inplace=True)

In [46]:
sdc_distribution = sdc_unique[['identifier','distribution']].explode('distribution')
sdc_distribution.dropna(subset='distribution', inplace=True)
sdc_distribution['download_url'] = sdc_distribution['distribution'].apply(lambda x: x['downloadURL'] if 'downloadURL' in x else None)
sdc_distribution['access_url'] = sdc_distribution['distribution'].apply(lambda x: x['accessURL'] if 'accessURL' in x else None)
sdc_distribution['link_title'] = sdc_distribution['distribution'].apply(lambda x: x['title'] if 'title' in x else None)
sdc_distribution['link_desc'] = sdc_distribution['distribution'].apply(lambda x: x['description'] if 'description' in x else None)
sdc_distribution['media_type'] = sdc_distribution['distribution'].apply(lambda x: x['mediaType'] if 'mediaType' in x else None)
sdc_distribution.drop(columns="distribution", inplace=True)
sdc_distribution['access_url'] = sdc_distribution['access_url'].fillna('')

sdc_distribution['doi'] = sdc_distribution['access_url'].apply(lambda x: x.split('doi.org/')[-1] if 'doi.org/' in x else None)

In [68]:
dois = sdc_distribution[['identifier','doi']].dropna(subset='doi').reset_index(drop=True)

dois['doi'] = dois['doi'].apply(lambda x: x.split(':')[-1] if ':' in x else x)
dois['doi'] = dois['doi'].apply(lambda x: f"10.50{x}" if x.startswith('066/') else x)
dois['doi'] = dois['doi'].apply(lambda x: x[1:] if x.startswith('/') else x)
dois['doi'] = dois['doi'].apply(lambda x: x.replace('10/5066', '10.5066') if x.startswith('10/5066') else x)
dois = dois[(dois['doi'].str.startswith('10.')) & (~dois['doi'].str.endswith('xxxx'))]

In [79]:
access_links = sdc_distribution[
    (sdc_distribution['link_title'] == 'Digital Data')
    &
    (~sdc_distribution['access_url'].str.contains('doi.org'))
    &
    (sdc_distribution['access_url'].str.len() > 0)
][['identifier','access_url']].reset_index(drop=True)
access_links = access_links[access_links['access_url'].str.startswith('http')]

In [88]:
download_links = sdc_distribution[
    (sdc_distribution['download_url'].str.len() > 0)
    &
    (sdc_distribution['link_title'] != 'Original Metadata')
][['identifier','download_url']].reset_index(drop=True)
download_links = download_links[download_links['download_url'].str.startswith('http')]

In [101]:
sdc_contacts = sdc_unique[sdc_unique['contactPoint'].notnull()][['identifier','contactPoint']].reset_index(drop=True)
sdc_contacts['name'] = sdc_contacts['contactPoint'].apply(lambda x: x['fn'])
sdc_contacts['email'] = sdc_contacts['contactPoint'].apply(lambda x: x['hasEmail'])
sdc_contacts['email'] = sdc_contacts['email'].apply(lambda x: x.replace('mailto:', '') if x is not None else None)
sdc_contacts.drop(columns='contactPoint', inplace=True)
sdc_contacts.dropna(subset='email', inplace=True)

In [1]:
sdc_unique[sdc_unique['title'].str.len() > 250]

NameError: name 'sdc_unique' is not defined